In [30]:
from utils import *
import re
from pprint import pprint
from nltk.corpus import stopwords

all_data = get_data_from_db()
all_data = all_data[(all_data["labels"]== "PO") | (all_data["labels"]== "NG")]

num_remover = NumRemover()
word_remover = WordRemover(stopwords.words('portuguese'))
all_data = num_remover.fit_transform(all_data)
all_data = word_remover.fit_transform(all_data)

neg_data = all_data[all_data["labels"] == "NG"]
pos_data = all_data[all_data["labels"] == "PO"]

neg_train = neg_data.iloc[0:round(0.3*neg_data.shape[0]), :]
pos_train = pos_data.iloc[0:round(0.3*pos_data.shape[0]), :]

neg_test = neg_data.iloc[round(0.3*neg_data.shape[0]): , :]
pos_test = pos_data.iloc[round(0.3*neg_data.shape[0]): , :]

stopwords_pt = stopwords.words("portuguese")

# Vocabulários

In [31]:
from nltk import FreqDist
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer

vocab_neg = set(' '.join(neg_train["texts"].tolist()).split())
vocab_pos = set(' '.join(pos_train["texts"].tolist()).split())

print("Dimensão vocabulario negativo: " + str(len(vocab_neg)))
print("Dimensão vocabulario positivo: " + str(len(vocab_pos)))

Dimensão vocabulario negativo: 1435
Dimensão vocabulario positivo: 1362


In [32]:
# Verificar interseção dos vocabulários
intersect = vocab_neg.intersection(vocab_pos)
vocab_neg = vocab_neg.difference(intersect)
vocab_pos = vocab_pos.difference(intersect)

lexicon = list(vocab_neg.union(vocab_pos))

In [33]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

features = FeatureUnion([
                    ("bigram", CountVectorizer(ngram_range=(2,2), stop_words= stopwords_pt, binary= True)),
                    ("lexicon_vector", CountVectorizer(vocabulary= lexicon))
                    ])

print("MultinomialNB")
run_cross_validation(all_data, features, MultinomialNB())

print("\n\nSVM")
run_cross_validation(all_data, features, SVC(C=316))

MultinomialNB
Cross Validation:
Fold  0  - Acuracia:  0.802816901408
Fold  1  - Acuracia:  0.771428571429
Fold  2  - Acuracia:  0.771428571429
Fold  3  - Acuracia:  0.871428571429
Fold  4  - Acuracia:  0.828571428571
Fold  5  - Acuracia:  0.814285714286
Fold  6  - Acuracia:  0.757142857143
Fold  7  - Acuracia:  0.742857142857
Fold  8  - Acuracia:  0.7
Fold  9  - Acuracia:  0.657142857143

Accuracia media:  0.771710261569
Desvio padrão:  0.0593886757314


SVM
Cross Validation:
Fold  0  - Acuracia:  0.774647887324
Fold  1  - Acuracia:  0.828571428571
Fold  2  - Acuracia:  0.742857142857
Fold  3  - Acuracia:  0.771428571429
Fold  4  - Acuracia:  0.828571428571
Fold  5  - Acuracia:  0.771428571429
Fold  6  - Acuracia:  0.742857142857
Fold  7  - Acuracia:  0.642857142857
Fold  8  - Acuracia:  0.657142857143
Fold  9  - Acuracia:  0.6

Accuracia media:  0.736036217304
Desvio padrão:  0.0739101033911
